In [2]:
import torch
from tqdm.auto import tqdm
from tqdm import trange
from transformers import AutoModel, AutoConfig, AutoTokenizer
from datasets import load_from_disk
import numpy as np

import random
import torch
import torch.nn.functional as F
from transformers import (
    XLMRobertaModel, AdamW, TrainingArguments, 
    get_linear_schedule_with_warmup, RobertaModel
)

from torch.utils.data import (
    DataLoader,
    RandomSampler,
    TensorDataset
)


# 데이터셋 및 토크나이저 불러오기

In [3]:
dataset = load_from_disk('/opt/ml/input/data/data/train_dataset')
model_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 토크나이저

In [4]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [6]:
tokenized_input = tokenizer(dataset['train'][0]['context'], padding='max_length', truncation=True)
tokenized_input

{'input_ids': [101, 23545, 9414, 10459, 14279, 20625, 23545, 9414, 14279, 113, 10609, 10859, 22294, 114, 9632, 9543, 14279, 17730, 12030, 23545, 9637, 56356, 9414, 73295, 11903, 119, 165, 182, 165, 182, 22458, 20479, 9365, 43022, 44220, 10739, 9414, 74125, 55635, 22143, 119, 8844, 9689, 21928, 123, 45441, 9414, 74125, 73295, 9428, 52363, 16855, 10407, 45441, 9414, 74125, 78686, 8908, 103764, 11506, 119, 9644, 46216, 127, 10954, 33542, 117, 92360, 101814, 10462, 21789, 16323, 9694, 122, 120, 124, 119108, 9414, 74125, 51684, 9415, 11261, 9428, 52363, 13374, 9568, 42337, 10530, 9356, 75884, 11903, 119, 165, 182, 165, 182, 22458, 20479, 9414, 84467, 23545, 9952, 14279, 11882, 11018, 9056, 78131, 23545, 70672, 10622, 9460, 30134, 11467, 23969, 23545, 9568, 42337, 9966, 16605, 52961, 8844, 22200, 9095, 78505, 23969, 8932, 20595, 11925, 119, 9952, 73295, 9435, 40032, 11882, 8885, 17730, 10530, 18154, 8917, 11102, 117, 70672, 10622, 104431, 9069, 11903, 57138, 8896, 32537, 51684, 9901, 14867, 

In [5]:
# sample_idx = np.random.choice(range(len(dataset['train'])), 128)
# training_dataset = dataset['train'][sample_idx]
# print(len(dataset['train']), len(training_dataset['title']))
training_dataset = dataset['train']

In [6]:
q_seqs = tokenizer(training_dataset['question'], padding='max_length', truncation=True, return_tensors='pt')
p_seqs = tokenizer(training_dataset['context'], padding='max_length', truncation=True, return_tensors='pt')

In [7]:
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'],
                             q_seqs['input_ids'], q_seqs['attention_mask'])

In [8]:
class RobertaEncoder(XLMRobertaModel):
    def __init__(self, config):
        super(RobertaEncoder, self).__init__(config)
        
        self.roberta = XLMRobertaModel(config)
        self.init_weights()
    
    def forward(self, input_ids, attention_mask=None):
        outputs = self.roberta(input_ids, attention_mask)
        pooled_output = outputs[1]
        return pooled_output

In [9]:
p_encoder = RobertaEncoder.from_pretrained(model_name)
q_encoder = RobertaEncoder.from_pretrained(model_name)

if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()
    print('GPU enabled')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing RobertaEncoder: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaEncoder were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['embeddings.word_embeddings.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.LayerNorm.weight', 'embeddin

GPU enabled


In [10]:
def train(args, dataset, p_model, q_model):
    
    # Dataloader
    train_sampler = RandomSampler(dataset)
    train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)
    
    # Optimizer
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay' : args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay' : 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay' : args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if  any(nd in n for nd in no_decay)], 'weight_decay' : 0.0},       
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
    
    global_step = 0
    
    p_model.zero_grad()
    q_model.zero_grad()
    torch.cuda.empty_cache()
    
    train_iterator = trange((int(args.num_train_epochs)), desc='Epoch')
    
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc='Iteration')
        
        for step, batch in enumerate(epoch_iterator):
            q_encoder.train()
            p_encoder.train()
            
            if torch.cuda.is_available():
                batch = tuple(t.cuda() for t in batch)
                
            p_inputs = {
                'input_ids' : batch[0],
                'attention_mask' : batch[1]
            }
            
            q_inputs = {
                'input_ids' : batch[2],
                'attention_mask' : batch[3]
            }
            
            p_outputs = p_model(**p_inputs)
            q_outputs = q_model(**q_inputs)
            
            # Calculate Similarity
            sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))
            
            # target
            targets = torch.arange(0, args.per_device_train_batch_size).long()
            if torch.cuda.is_available():
                targets = targets.to('cuda')
            
            sim_scores = F.log_softmax(sim_scores, dim=1)
            
            loss = F.nll_loss(sim_scores, targets)
            
            loss.backward()
            optimizer.step()
            scheduler.step()
            q_model.zero_grad()
            p_model.zero_grad()
            global_step += 1
            
            torch.cuda.empty_cache()

            
    return q_model, p_model

In [11]:
args = TrainingArguments(
    output_dir='dense_retrieval',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=40,
    num_train_epochs=2,
    weight_decay=0.01
)

In [ ]:
p_encoder, q_encoder = train(args, train_dataset, p_encoder, q_encoder)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
valid_corpus = list(dict.fromkeys(example['context'] for example in dataset['validation']))[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']
print(query, ground_truth)

미국 남부 지방에서 급격하게 영국산 제품이 많이 팔린 계기가 된 조치는? 통상금지법(Embargo Act of 1807)은 1807년에 미국 의회에서 통과된 법률이며 , 나폴레옹 전쟁을 치루고 있는 영국과 프랑스에 대항하여 미국의 무역을 금지한 것이었다\n\n이 통상금지(이하, 엠바고)는 미국의 상인과 화물선이 유럽의 해군에게 금수품으로 나포하면서, 미국의 중립 위반에 대한 보복 조치를 취하게 되자 이 법안을 시행하게 되었다. 영국 해군은 특히 수천 명의 미국 선원들을 자신들의 함대에 복역하게 하는 강제 징발 조치를 감행하였다. 유럽에 대한 통제권을 위해 필사적으로 다투었던 영국과 프랑스는 우발적인 사고이고, 그들의 생존을 위해 필수적인 조치였다고 합리화했다. 미국인들은 체사피크 레오퍼드 사건을 중립을 지키는 미국의 조치에 대한 영국의 어처구니 없는 위반 사례라고 보았다 외교적 모욕이라고 이해를 하고, 유럽 열강에 의해 이러한 작전들을 지지하는 보증되지 않은 공식적인 명령서는 미국의 선전포고에 대한 근거로서 널리 인정되고 있다. 노한 군중들은 냉소적으로 ‘엠바고’(Embargo)라는 글자를 거꾸로 ‘나 좀 잡아줘’(O Grab Me)라고 바꿔 불렀다. \n\n토머스 제퍼슨 대통령은 이러한 분노한 대중들의 적의를 등에 업고, 보복에 대한 대중적 지지에 힘을 싣고 있었다. 그는 의회에 군사적으로 물리적 조치보다는 경제 전쟁으로 응전해야 한다고 권했다. 그리하여 통상금지법이 1807년 12월 22일 서명되었다. 이 급작스런 조치에 대한 기대 효과(교전국 사이의 경제적 곤란 )는 영국과 프랑스를 힘들게 하여, 그들로 하여금 미국 상선을 방해를 하지못하도록 강제하고, 미국의 중립을 존중하며, 강제 징발 정책을 끝내는 것이었다. 이 엠바고는 강압적인 조치로서는 그다지 효과를 보지 못한 것으로 밝혀졌으며, 외교적으로도, 경제적으로 모두 실패한 조치였다. 이 법이 막상 시행되고 나서, 미국 경제와 미국 국민들에게 파괴적인 부담을 주었다. \n\n미국 상인들에 의한 바다와 내

In [19]:
if not ground_truth in valid_corpus:
    valid_corpus.append(ground_truth)

In [20]:
def to_cuda(batch):
    return tuple(t.cuda() for t in batch)

In [31]:
with torch.no_grad():
    p_encoder.eval()
    q_encoder.eval()
    
    q_seqs_val = tokenizer([query], padding='max_length', truncation=True, return_tensors='pt').to('cuda')
    q_emb = q_encoder(**q_seqs_val).to('cpu')
    
    p_embs = []
    for p in valid_corpus:
        p = tokenizer(p, padding='max_length', truncation=True, return_tensors='pt').to('cuda')
        p_emb = p_encoder(**p).to('cpu').numpy()
        p_embs.append(p_emb)
    
    p_embs = torch.Tensor(p_embs).squeeze()
    
    print(p_embs.shape, q_emb.shape)

torch.Size([11, 768]) torch.Size([1, 768])


In [33]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.shape)

torch.Size([1, 11])


In [34]:
rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(dot_prod_scores)
print(rank)

tensor([[86.5999, 86.6622, 86.6466, 86.6187, 86.6514, 86.5946, 86.5708, 86.6117,
         86.6401, 86.6131, 86.6357]])
tensor([ 1,  4,  2,  8, 10,  3,  9,  7,  0,  5,  6])
